In [1]:
#pip install -U jupyter
#pip install --upgrade tomotopy
#pip install --upgrade pandas
#pip install reshape
#pip install gensim
#pip install soynlp

import re
import gensim₩
import numpy as np
import tomotopy as tp
import tomotopy.coherence
from kiwipiepy import Kiwi, Option
from pyvis.network import Network

In [2]:
kiwi = Kiwi()

In [3]:
Kiwi(num_workers=-0, model_path='./', options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH)

In [4]:
try:
    corpus = tp.utils.Corpus.load('k.cps')
except IOError:
    kiwi = Kiwi()
    kiwi.add_user_word('전달력', tag='NNG')
    kiwi.add_user_word('영상미', tag='NNG')
    kiwi.add_user_word('영상기법', tag='NNG')
    kiwi.add_user_word('영상구도', tag='NNG')
    kiwi.add_user_word('배경음악', tag='NNG')
    kiwi.add_user_word('구독자수', tag='NNG')   
    kiwi.add_user_word('레고프렌즈', tag='NNG')
    kiwi.add_user_word('딱히', tag='MAG')
    kiwi.add_user_word('교육', tag='NNP')
    kiwi.add_user_word('방송', tag='NNP')
    kiwi.add_user_word('브이로그', tag='NNP')
    kiwi.add_user_word('생일', tag='NNP')
    kiwi.add_user_word('날', tag='NNP')
    kiwi.add_user_word('전반적', tag='NNP')
    kiwi.add_user_word('조회수', tag='NNP')
    kiwi.add_user_word('반려견', tag='NNP')
    kiwi.add_user_word('매니아층', tag='NNP')
    kiwi.add_user_word('생방송', tag='NNP')
    kiwi.add_user_word('먹방', tag='NNP')
    kiwi.add_user_word('배경음악', tag='NNP')
    kiwi.add_user_word('맛집', tag='NNP')
    kiwi.add_user_word('재건축', tag='NNP')
    kiwi.add_user_word('언박싱', tag='NNP')    
    kiwi.prepare()
    pat_tag = re.compile('NN[GP]|V[VA]|MAG|MM') 

In [5]:
    def tokenizer(raw, user_data):
        res, _ = user_data()[0]
        for w, tag, start, l in res:
            if not pat_tag.match(tag) or len(w) <= 1: continue
            yield w + ('다' if tag.startswith('V') else ''), start, l

In [6]:
    corpus = tp.utils.Corpus(
        tokenizer=tokenizer
    )
    
    corpus.process((line, kiwi.async_analyze(line)) for line in open('ctm_data_final_210322.txt', encoding='UTF-8'))
    corpus.save('data_210322_1.cps')

In [7]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

In [8]:
#PMI 산출 코드
#def getPMI(self, a, b):
    #import math
    #co = self.dictNear.get((a,b),0)
    #if not co: return None
    #return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])

case1. tw기본값 (tw=tp.TermWeight.ONE)으로 세팅했을 때!

In [9]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

#아래는 코드 (#지우기)
#mdl = tp.CTModel(tw=tp.TermWeight.ONE, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
#mdl.train(0)

case2. tw에 PMI로 가중치 부여했을 때

In [10]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

#아래는 코드 (#지우기)
#mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
#mdl.train(0)

case3. tw에 PMI로 가중치 부여하고, min_df=0, min_cf=0로 설정 (개발자분 피드백)

In [11]:
# 최소 0개 이상 문헌에 등장하고, 전체 출현빈도는 0 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 10개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=0, min_cf=0, rm_top=0, k=10, corpus=corpus)
mdl.train(0)

In [12]:
#num_beta_sample은 만 개 이상이면 1~5, 수 천개라면 최소 10, 수 백개에 불과하면 20이상으로 키우기
mdl.num_beta_sample = 10
print('Num docs:{}, Total Words:{}'.format(
    len(mdl.docs), mdl.num_words
))

Num docs:147, Total Words:465


In [13]:
#깁스샘플링
for i in range(0, 1000, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)

Iteration: 0000, LL per word: -27.3
Iteration: 0020, LL per word: -25.27
Iteration: 0040, LL per word: -26.17
Iteration: 0060, LL per word: -26.94
Iteration: 0080, LL per word: -27.48
Iteration: 0100, LL per word: -27.88
Iteration: 0120, LL per word: -29.26
Iteration: 0140, LL per word: -29.23
Iteration: 0160, LL per word: -29.02
Iteration: 0180, LL per word: -27.78
Iteration: 0200, LL per word: -27.4
Iteration: 0220, LL per word: -27.74
Iteration: 0240, LL per word: -27.45
Iteration: 0260, LL per word: -27.98
Iteration: 0280, LL per word: -27.85
Iteration: 0300, LL per word: -28.44
Iteration: 0320, LL per word: -27.64
Iteration: 0340, LL per word: -28.08
Iteration: 0360, LL per word: -28.89
Iteration: 0380, LL per word: -29.14
Iteration: 0400, LL per word: -27.86
Iteration: 0420, LL per word: -27.79
Iteration: 0440, LL per word: -27.62
Iteration: 0460, LL per word: -27.13
Iteration: 0480, LL per word: -27.03
Iteration: 0500, LL per word: -27.91
Iteration: 0520, LL per word: -27.66
Ite

In [14]:
print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))

Iteration: 0980, LL per word: -27.19


In [15]:
mdl.summary()

#Words: CTM 분석 후 추출된 문서들 전체의 단어의 개수 (문서 당 같은 단어가 2개 있을 수도 있고 중복 가능)
#Total Vocabs: 작은 빈도의 단어들을 제거한 뒤 남은 토픽모델링 후 어휘의 개수
#Used_Vocabs: 모델에 실제로 사용된 어휘만을 포함하는 dictionary 타입의 어휘 사전
#prior_cov : 토픽의 사전분포인 로지스틱 정규분포의 공분산 행렬
#prior_mean : 토픽의 사전분포인 로지스틱 정규분포의 평균 벡터
#k : 토픽의 개수, 1 ~ 32767 범위의 정수.
#alpha는 CTM 모형의 모수 
#eta는 모수 
#num.iterations는 반복 횟수로 z를 몇 번 추출하느냐


<Basic Info>
| CTModel (current version: 0.10.2)
| 147 docs, 465 words
| Total Vocabs: 273, Used Vocabs: 273
| Entropy of words: -5.33406
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 2
| Log-likelihood per word: -27.18626
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| smoothing_alpha: 0.1 (small smoothing value for preventing topic counts to be zero)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1505892886 (random seed)
| trained in version 0.10.2
|
<Parameters>
| prior_mean (Prior mean of Logit-normal for the per-document topic distributions)
|  [-3.916378  -3.7780397 -4.6356635 -3.5264432 -3.9766529 -4.304249
|   -3.922733  -4.221271  -3.4640088 -1.9515429]
| prior_cov (Prior covariance of Logit-n

In [16]:
mdl.save('trained_ctm_model.bin')

In [17]:
import itertools

In [18]:
for preset in ('u_mass', 'c_uci', 'c_npmi', 'c_v'):
    coh = tp.coherence.Coherence(mdl, coherence=preset)
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence : {} ===='.format(preset))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

==== Coherence : u_mass ====
Average: -22.29408313226251 
Per Topic: [-21.736088398794525, -25.254564104263448, -25.248171169319075, -17.10559745096002, -19.903030660140622, -21.25052667434378, -23.559608767788863, -23.454930307758826, -22.96881647130552, -22.45949731795046]

==== Coherence : c_uci ====
Average: -14.891648000479142 
Per Topic: [-14.742224178774192, -16.702203446382878, -17.003018922511124, -11.965786918676798, -13.477880959211687, -13.757370245808978, -15.733985850494216, -14.718951094549489, -15.272843115715348, -15.542215272666704]

==== Coherence : c_npmi ====
Average: -0.44048706080386746 
Per Topic: [-0.4187020790447674, -0.5518828529882321, -0.562148778471677, -0.27206945534076094, -0.33245849503554586, -0.38833396183926633, -0.49762467620861106, -0.4278069234224893, -0.4663166815891999, -0.4875267040981248]

==== Coherence : c_v ====
Average: 0.9290446770191192 
Per Topic: [0.9023079097270965, 0.9449620604515075, 0.9417855381965637, 0.9252568304538726, 0.9216608

In [19]:
for seg, cm, im in itertools.product(tp.coherence.Segmentation, tp.coherence.ConfirmMeasure, tp.coherence.IndirectMeasure):
    coh = tp.coherence.Coherence(mdl, coherence=(tp.coherence.ProbEstimation.DOCUMENT, seg, cm, im))
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence : {}, {}, {} ===='.format(repr(seg), repr(cm), repr(im)))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.NONE: 0> ====
Average: 0.07581065758864243 
Per Topic: [0.07329931972248091, 0.04565381707494779, 0.0378117913772651, 0.11155706726558451, 0.13718820860598194, 0.08887944065542393, 0.05163265305438445, 0.10275888131592738, 0.05925359031850517, 0.05007180649592312]

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.COSINE: 1> ====
Average: 0.08248412334748234 
Per Topic: [0.07983918136192693, 0.04259674313167731, 0.06664826898421679, 0.16974862636610244, 0.1291230795232372, 0.16656952985148463, 0.03827623688201937, 0.04370214407875513, 0.0566501791537222, 0.03168724414168133]

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.DICE: 2> ====
Average: 4.748609274898966 
Per Topic: [20.541754910680982, 1.5022152794731989, 0.5279186328252157, 3.6042629096243117, -1.7228050820529461, 1.99129691918691, 5.7296934352980

==== Coherence : <Segmentation.ONE_PRE: 2>, <ConfirmMeasure.RATIO: 2>, <IndirectMeasure.DICE: 2> ====
Average: 0.05550633029184406 
Per Topic: [0.06661618190507095, 0.02979795957605044, 0.029834761677516832, 0.1064976824240552, 0.08968147304322985, 0.059551421801249185, 0.03537949487153027, 0.056310342914528314, 0.04130795862939623, 0.040086026075813505]

==== Coherence : <Segmentation.ONE_PRE: 2>, <ConfirmMeasure.RATIO: 2>, <IndirectMeasure.JACCARD: 3> ====
Average: 0.07711458843201399 
Per Topic: [0.08215603126833836, 0.05320647797650761, 0.04703999592198266, 0.14565196873413191, 0.12900501415133475, 0.07426099557843473, 0.04537087999698189, 0.08906451761722564, 0.053259435585803455, 0.05213056748939885]

==== Coherence : <Segmentation.ONE_PRE: 2>, <ConfirmMeasure.LIKELIHOOD: 3>, <IndirectMeasure.NONE: 0> ====
Average: 26331065762.317696 
Per Topic: [10.122222220296116, 44142101279.513985, 29327286467.965836, 16213151935.444675, 43537414975.80492, 17913832203.67022, 19546485261.80242

==== Coherence : <Segmentation.ONE_SUC: 3>, <ConfirmMeasure.LOGLIKELIHOOD: 4>, <IndirectMeasure.DICE: 2> ====
Average: 0.5402398407459259 
Per Topic: [0.5547961592674255, 0.5368674397468567, 0.5368674397468567, 0.5410807132720947, 0.5479734539985657, 0.5367944240570068, 0.53669673204422, 0.5425752401351929, 0.5367514491081238, 0.5319953560829163]

==== Coherence : <Segmentation.ONE_SUC: 3>, <ConfirmMeasure.LOGLIKELIHOOD: 4>, <IndirectMeasure.JACCARD: 3> ====
Average: 1.1885324120521545 
Per Topic: [1.246161937713623, 1.15920889377594, 1.15920889377594, 1.179032325744629, 1.2122594118118286, 1.1588685512542725, 1.0197890996932983, 1.1861518621444702, 1.158668041229248, 1.0231053829193115]

==== Coherence : <Segmentation.ONE_SUC: 3>, <ConfirmMeasure.PMI: 5>, <IndirectMeasure.NONE: 0> ====
Average: -17.774982745184246 
Per Topic: [-20.135062593959827, -19.44191541520063, -19.44191541520063, -20.135062593959827, -20.135062593959827, -19.44191541520063, -19.44191541520063, -19.4419154152006

==== Coherence : <Segmentation.ONE_ALL: 4>, <ConfirmMeasure.PMI: 5>, <IndirectMeasure.JACCARD: 3> ====
Average: 9.008780441284179 
Per Topic: [9.005330467224121, 9.0048752784729, 9.00417947769165, 9.014835262298584, 9.011392402648926, 9.00784969329834, 9.006060791015624, 9.001980876922607, 9.003767490386963, 9.02753267288208]

==== Coherence : <Segmentation.ONE_ALL: 4>, <ConfirmMeasure.NPMI: 6>, <IndirectMeasure.NONE: 0> ====
Average: 0.0 
Per Topic: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

==== Coherence : <Segmentation.ONE_ALL: 4>, <ConfirmMeasure.NPMI: 6>, <IndirectMeasure.COSINE: 1> ====
Average: 0.9099923872947695 
Per Topic: [0.8755637049674988, 0.9304057478904724, 0.9263676583766938, 0.9052387773990631, 0.9007247686386108, 0.9189103126525879, 0.905403059720993, 0.9321994304656982, 0.9201637208461761, 0.8849466919898987]

==== Coherence : <Segmentation.ONE_ALL: 4>, <ConfirmMeasure.NPMI: 6>, <IndirectMeasure.DICE: 2> ====
Average: 0.9022966295480728 
Per Topic: [0.90122

In [20]:
import matplotlib.pyplot as plt

In [21]:
import gensim

In [22]:
mdl.used_vocabs   

#Used Vocabs 의미 >> 78개
#242 words >> k(10)에 속한 총 단어의 수 (84개가 중복되어서 들어감)  == 총 단어의 빈도 수

['영상', '브이로그', '일상', '게임', '편집', '재미있다', '좋아하다', '만들다', '사람', '정보', '주제', '여행', '자유', '소통', '공유', '많이', '모습', '강아지', '대하다', '가장', '활동', '시청자', '도움', '관심', '패션', '생활', '콘텐츠', '업로드', '라이브', '보이다', '추억', '운동', '취미', '교육', '귀엽다', '분야', '관련', '처음', '방송', '실시간', '제작', '즐거움', '음악', '내용', '먹방', '촬영', '자연', '영상미', '고양이', '수업', '아이', '직장', '반려견', '즐기다', '함께', '노력', '직접', '연주', '노래', '지식', '전달', '기존', '리뷰', '위하다', '사회', '경험', '기록', '어리다', '공감', '나오다', '가족', '요리', '배경음악', '신경', '요소', '제품', '비교', '올리다', '사용', '다이어트', '새롭다', '메뉴', '딱히', '구독자수', '얽매이다', '저장', '건강', '차량', '시승', '전달력', '메추리', '실력', '다양', '접근성', '용이', '남기다', '온라인', '헬스장', '다른', '동료', '동호회', '구입', '골프', '연습', '방법', '환경', '정성', '들어가다', '자녀', '역사', '컨셉', '생각', '그저', '그대로', '산책', '해외', '소개', '패션쇼', '유명한', '밴드', '신규', '접하다', '겨울왕국', '라이', '특징', '후원', '채팅', '식물', '키우다', '실무', '동기', '동창', '리액션', '너무', '공감대', '다시', '지금', '학교', '과제', '시연', '스피드', '페인팅', '제공', '주식', '토크', '경제', '유투', '육아', '베이비', '특정', '자신', '특별', '매니아층', '토킹', '편하다', '과외', '영화',

In [23]:
#Visualize the result

g = Network(width=800, height=800, font_color="#333")
correl = mdl.get_correlations().reshape([-1])

In [24]:
correl.sort()

In [25]:
# 상관계수 상위 10%만 간선으로 잇고
top_tenth = mdl.k * (mdl.k - 1) // 10
top_tenth = correl[-mdl.k - top_tenth]

In [26]:
topic_counts = mdl.get_count_by_topics()

In [27]:
print(topic_counts)

#합하면 465 (242words) >> 빈도수 465

[53 33 39 56 49 41 44 24 52 74]


In [28]:
for k in range(mdl.k):
    label = "#{}".format(k)
    title= ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=8))
    print('Topic', label, title)
    label += '\n' + ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=3))
    g.add_node(k, label=label, title=title, shape='ellipse', value=float(topic_counts[k]))
    for l, correlation in zip(range(k - 1), mdl.get_correlations(k)):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))

Topic #0 공유 생활 내용 가장 취미 처음 기록 직장
Topic #1 재미있다 교육 어리다 촬영 요소 베이킹 영화 생방송
Topic #2 패션 관심 즐거움 업로드 공감 먹방 가족 스타일
Topic #3 소통 좋아하다 자유 활동 시청자 음악 많이 실시간
Topic #4 만들다 추억 분야 대하다 고양이 사용 요리 지식
Topic #5 브이로그 일상 강아지 귀엽다 반려견 나오다 개인 과외
Topic #6 여행 사람 주제 제작 수업 함께 로또 블로그
Topic #7 운동 콘텐츠 위하다 구입 장난감 다이어트 헬스장 언박싱
Topic #8 게임 정보 라이브 도움 즐기다 제품 비교 전달력
Topic #9 영상 편집 관련 보이다 모습 자연 직접 연주


In [1]:
#토픽 topic_id에 속하는 상위 top_n개의 단어와 각각의 확률을 반환.
#반환 타입은 (단어:str, 확률:float) 튜플의 list형
#확률은 현재 토픽 내 각각의 단어들의 발생확률?

topic_words = mdl.get_topic_words(k, top_n=100)

NameError: name 'mdl' is not defined

In [ ]:
import pandas as pd

In [ ]:
df_topic_words = pd.DataFrame(topic_words)

In [ ]:
#https://dandyrilla.github.io/2017-08-12/pandas-10min/

df_topic_words

In [33]:
df_topic_words.describe()

,1
count,50.000000
mean,0.019839
std,0.017741
min,0.000036
25%,0.011970
50%,0.019667
75%,0.023695
max,0.104379


In [34]:
topic_words

[('영상', 0.10437855124473572),
 ('편집', 0.06009669974446297),
 ('관련', 0.043095991015434265),
 ('보이다', 0.040451738983392715),
 ('모습', 0.035881705582141876),
 ('자연', 0.032842058688402176),
 ('직접', 0.032842058688402176),
 ('연주', 0.032842058688402176),
 ('영상미', 0.03138129785656929),
 ('리뷰', 0.03138129785656929),
 ('노력', 0.025203438475728035),
 ('배경음악', 0.02369496040046215),
 ('기존', 0.02369496040046215),
 ('신경', 0.02369496040046215),
 ('아직', 0.022163789719343185),
 ('캠핑', 0.022163789719343185),
 ('겨울왕국', 0.019666604697704315),
 ('스피드', 0.019666604697704315),
 ('동네', 0.019666604697704315),
 ('인천', 0.019666604697704315),
 ('실력', 0.019666604697704315),
 ('산책', 0.019666604697704315),
 ('페인팅', 0.019666604697704315),
 ('주식', 0.019666604697704315),
 ('토크', 0.019666604697704315),
 ('환경', 0.019666604697704315),
 ('동물', 0.01820584572851658),
 ('애착', 0.01820584572851658),
 ('신규', 0.01820584572851658),
 ('홍보', 0.01820584572851658),
 ('모든', 0.01820584572851658),
 ('유명한', 0.017169421538710594),
 ('연습', 0.0

In [35]:
g.barnes_hut(gravity=-1000, spring_length=20)

In [36]:
g.show_buttons()
g.show("topic_network_210318_2.html")

In [37]:
import pyLDAvis
import pyLDAvis.gensim

/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-no

In [38]:
pyLDAvis.enable_notebook()

In [39]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])

In [40]:
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims= True)

In [41]:
prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
    
)

In [42]:
pyLDAvis.display(prepared_data)

In [46]:
htmlFile = 'tomotopy_CTM TO LDA_210322_2.html'
pyLDAvis.save_html(prepared_data, htmlFile)

In [44]:
from bs4 import BeautifulSoup
with open(htmlFile) as inf:
    txt = inf.read()
    soup = BeautifulSoup(txt)

In [45]:
print(soup)

<html><head><link href="https://cdn.jsdelivr.net/gh/bmabey/pyLDAvis@3.2.2/pyLDAvis/js/ldavis.v1.0.0.css" rel="stylesheet" type="text/css"/>
</head><body><div id="ldavis_el754321404274581657763115694940"></div>
<script type="text/javascript">

var ldavis_el754321404274581657763115694940_data = {"mdsDat": {"x": [-0.31691198362938117, 0.12399059825376554, -0.06661567701578111, 0.11810085834315132, 0.10864693453153175, 0.09971817040633538, -0.26510549162639874, 0.04536340558869734, 0.08595669604126824, 0.06685648910681108], "y": [0.04154598794899229, 0.09458320090132395, -0.296133558423676, 0.0804994893894477, 0.07293033093550291, 0.06824893383150327, 0.1390620129354153, -0.30823181564048785, 0.05954195870996102, 0.047953459412017146], "topics": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "cluster": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "Freq": [15.109658318493535, 12.0335451387713, 11.96926930206475, 10.996503972801655, 10.734924840504318, 9.60231514556859, 8.51076935272928, 8.351612966004634, 7.468123001